#                         Assignment 1 - Task 1 Parsing Raw Text Files

### Student Name: Roopak TJ
### Student ID: 29567467

Date: 14th April, 2019

Environment: Python 3 and Jupyter notebook
Libraries used: 
* re - for regular expressions 
* json - package to write to json

External files used:
* Input file : 29567467.txt
* Output file : 29567467.xml | 29567467.json

# Introduction

Task includes analyzing of textual data, i.e., extracting data from semi-structured text files. Given file contains
information about several units in the Monash University. Each data-set contains information about the unit, e.g., unitcode, unit title, synopsis, requirements, output, chief examiner, etc. 





## Imports

In [4]:
import re
import json

## Overall approach

A class UnitDetailsExtraction is implemented which in a nutshell is fed with the .txt file having the unit information and it returns valid .xml and .json files.

functions implemented in the class:
* init function: This is the initialization function(constructor) which takes the input .txt file, reads the file and calls other function unitDivision which will divide the file
* unitDivision function: The function takes the complete input as string and uses regex function to divide the string into a list containing different unit code informations. After extracting each unit in a list, the list is iterated and extractUnitInfo function is called for individual unit information string. Create Json function is also called at this moment which will generated the json dictionary sequentially. After iteration of the whole string is completed, writexml and writejson functions are called which will write to file in the .xml and .json format respectively. 
* extractUnitInfo function: This function takes a string containing information of a single unit. Different regex functions are written to extract unit code, title, synopsis, pre-requisites, prohibitors, outcomes, requirements, and examiners. The data is stored in the corressponding class variable and will be utilised by other functions for this instance. createXml function is called at the end of this function.
* createXml function will use the data of the instance to append to a string in the xml file format
* createJson function will be used to create a ditionary to write as a json file
* writeJson function will write the dictionary to a json file using the json package after complete iteration
* writeXml function will write the string to an XML file after complete iteration of the unit list

In [5]:
# Class
class UnitDetailsExtraction:
    
    # class variables
    unitCode = ""
    title = ""
    prohibitors = []
    synopsis = []
    synopsis2 = []
    prequisites = []
    outcomes = []
    outcomes2 = []
    examiner = []
    requirements = []
    requirements2 = []
    xmlString = "<?xml version=\"1.0\" encoding=\"UTF-8\" ?>"
    xmlString = xmlString + "\n<units>\n"
    # Dictionary
    unit = {}
    unit['units'] = []
    unit_list = []
    
    
    # This is the initialization function(constructor) which takes the input .txt file, reads the 
    # file and calls other function unitDivision which will divide the file
    def __init__(self, filepath):
        text = open(filepath, mode= 'r')
        # converts the file object to a string
        fullUnit = text.read()
        self.unitDivision(fullUnit)
     
    
    # The function takes the complete input as string and uses regex function to divide the 
    # string into a list containing different unit code informations
    def unitDivision(self, fullUnit):
        unitList=re.findall(r"unitcode\">.*?content_container-->", fullUnit, re.DOTALL)
        # Create a list to all units
        #just passing one unit to test
        
        for each in unitList:
            # File has many string "&amp". Substituting it to "&#38;" as performed in the standard test.xml
            each = re.sub(r"&amp;", "&#38;", each)
            self.extractUnitInfo(each)
            self.createJson()
        
        # After complete iteration of the unit list, class variables will be populated with the required data.
        # Using writeXML and writeJson function to write to .xml and .json files respectively
        self.writeXML()
        self.writeJson()
    
    # extractUnitInfo function: This function takes a string containing information of a single unit. Different 
    # regex functions are written to extract unit code, title, synopsis, pre-requisites, prohibitors, outcomes, requirements, and examiners.
    def extractUnitInfo(self, unit_string):
        #Extracting Unit Code
        self.unit_code = re.findall(r"code[^<]+", unit_string)[0][6:]
        
        # Have identified some cases when unit code has more than 4 alphabets, only taking 3 alphabets
        if (len(self.unit_code) > 7):
            self.unit_code = self.unit_code[-7:]
        
        self.prohibitors = []
        self.synopsis = []
        self.prequisites = []
        self.outcomes = []
        self.examiner = []
        self.requirements = []
        
        # Extracting title
        self.title = re.findall(r"pan>.*?<sp", unit_string)[0][6:-3]

        # Extracting Prohibitors
        b = re.findall(r"Prohibitions</p>.*?</div", unit_string, re.DOTALL)

        if len(b) > 0:
            # Have identified some cases when unit code has more than 4 alphabets, only taking 3 alphabets
            c = re.findall(r"[A-Z]{3,4}\d{4}", b[0], re.DOTALL)
            if len(c) > 0:
                for each in c:
                    if (len(each) > 7):
                        each = each[-7:]
                    self.prohibitors.append(each)
        # Soring the prohibitors list to incresing matching to the test.xml(standard)
        self.prohibitors = sorted(list(set(self.prohibitors)))

        # Synopsis
        s = re.findall(r"Synopsis</h2>.*?</div", unit_string, re.DOTALL)
        
        # Taking just the first synopsis from the list.
        if len(s) > 0:
            s1 = re.findall(r"<p>.*?</p>", s[0], re.DOTALL)
            if len(s1) > 0:
                self.synopsis.append(s1[0][3:-4])
                
                
        # Identified that synopsis also contains words like <a, <p, <ol. This can 
        # make the XML invalid if not removed properly
        
        self.synopsis2 = []
        for each in self.synopsis:
            rem = re.findall(r"<a.*?>|</a>|</span>|<span.*?>", each, re.DOTALL)
            for r in rem:
                each = re.sub(r,"",each)
            self.synopsis2.append(each)
        self.synopsis = self.synopsis2
                
        # Extracting Prequisites
        p = re.findall(r"Prerequisites</p>.*?hbk-preamble-heading\">Prohibitions", unit_string, re.DOTALL)
        if len(p) == 0:
            p = re.findall(r"Prerequisites</p>.*?</div>", unit_string, re.DOTALL)

        # Have identified some cases when unit code has more than 4 alphabets, only taking 3 alphabets
        if len(p) > 0:
            p1 = re.findall(r"[A-Z]{3,4}\d{4}", p[0], re.DOTALL)
            if len(p1) > 0:
                for each in p1:
                    if (len(each) > 7):
                        each = each[-7:]
                    self.prequisites.append(each)
        
        if len(p) == 0:
            # Corequisite also needs to be added to Prerequisite list
            
            c = re.findall(r"Co-requisites</p>.*?hbk-preamble-heading\">Prohibitions", unit_string, re.DOTALL)
            if len(c) == 0:
                c = re.findall(r"Co-requisites</p>.*?</div>", unit_string, re.DOTALL)

            if len(c) > 0:
                c1 = re.findall(r"[A-Z]{3,4}\d{4}", c[0], re.DOTALL)
                if len(c1) > 0:
                    for each1 in c1:
                        if (len(each) > 7):
                            each = each[-7:]
                            self.prequisites.append(each1)
        
        # Soring the prohibitors list to incresing matching to the test.xml(standard)
        self.prequisites = sorted(list(set(self.prequisites)))
        
        # Identified that UnitCode with prefix "LAW" has no prerequisite field in the final output
        if "LAW" in self.unit_code:
            self.prequisites = []

        # Extracting Outcomes
        o = re.findall(r"Outcomes</h2>.*?</div", unit_string, re.DOTALL)

        if len(o) > 0:
            o1 = re.findall(r"<li>.*?</li>", o[0], re.DOTALL)
            if len(o1) > 0:
                for each in o1:
                    self.outcomes.append(each[4:-5])
                    
        # Identified that synopsis also contains words like <a, <p, <ol. This can 
        # make the XML invalid if not removed properly
        self.outcomes2 = []
        for each in self.outcomes:
            rem = re.findall(r"<a.*?>|</a>|</span>|<span.*?>|</?p>|</?ul>|</?li>|<ol.*?>|</ol>", each, re.DOTALL)
            for r in rem:
                each = re.sub(r,"",each)
            self.outcomes2.append(each)
        self.outcomes = self.outcomes2

        # Chief Examiner
        c = re.findall(r"examiner.*?class=", unit_string, re.DOTALL)

        if len(c) > 0:
            # Check if we have other salutations
            c1 = re.findall(r">.+?</a>",c[0])
            if len(c1) > 0:
                for each in c1:
                    self.examiner.append(each[1:-4])

        # Workload Requirements is taken from the assessment section of the unit information
        d = re.findall(r"ssment</h2>.*?</div>", unit_string, re.DOTALL)

        if len(d) > 0:
            # Check if we have other salutations
            d1 = re.findall(r"<p>.*?</p>", d[0], re.DOTALL)
            if len(d1) > 0:
                for each in d1:
                    self.requirements.append(each[3:-4])
                      
        self.requirements2 = []
        for each in self.requirements:
            rem = re.findall(r"<a.*?>|</a>|</span>|<span.*?>", each, re.DOTALL)
            for r in rem:
                each = re.sub(r,"",each)
            self.requirements2.append(each)
        self.requirements = self.requirements2
        
        # After extracting information for a unit, createXml function is called which 
        # will write the xml in the proper format into a string. Every iteration the 
        # string will be appended
        self.createXml()
        
    # Creating an XML file       
    def createXml(self):
        
        # For all the fields, existing value from the class variable is appended 
        # with the new value of the instance
        
        self.xmlString = self.xmlString + "<unit id='" + self.unit_code + "'>\n" + \
        "<title>" + self.title + "</title>\n"
        
        synopsisString = "<synopsis> "
        
        # If synopsis is empty, NA will be added to the corressponding XML value
        if len(self.synopsis) > 0:
            for each in self.synopsis:
                synopsisString = synopsisString + each
        else:
            synopsisString = synopsisString + " NA "
            
        synopsisString = synopsisString + "</synopsis>\n"
        
        self.xmlString = self.xmlString + synopsisString
        
        
        
        # If preRequisite is empty, NA will be added to the corressponding XML value

        preRequisiteString = "<pre_requistics>"
        
        if len(self.prequisites) > 0:
            preRequisiteString += "\n"
            for each in self.prequisites:
                preRequisiteString = preRequisiteString +"<pre_requistic>" +each+"</pre_requistic>"
        else:
            preRequisiteString = preRequisiteString + " NA "
            
        preRequisiteString = preRequisiteString + "</pre_requistics>\n"
        
        self.xmlString = self.xmlString + preRequisiteString
        
        
        prohibisionsString = "<prohibisions>"
        
        if len(self.prohibitors) > 0:
            prohibisionsString += "\n"
            for each in self.prohibitors:
                prohibisionsString = prohibisionsString +"<prohibision>" +each+"</prohibision>"
        else:
            prohibisionsString = prohibisionsString + " NA "
            
        prohibisionsString = prohibisionsString + "</prohibisions>\n"
        
        self.xmlString = self.xmlString + prohibisionsString
        
        
        requirementsString = "<requirements>"
        
        if len(self.requirements) > 0:
            requirementsString += "\n"
            for each in self.requirements:
                requirementsString = requirementsString +"<requirement>" +each+"</requirement>"
        else:
            requirementsString = requirementsString + " NA "
            
        requirementsString = requirementsString + "</requirements>\n"
        
        self.xmlString = self.xmlString + requirementsString
        
        
        outcomesString = "<outcomes>"
        
        if len(self.outcomes) > 0:
            outcomesString += "\n"
            for each in self.outcomes:
                outcomesString = outcomesString +"<outcome>" +each+"</outcome>"
        else:
            outcomesString = outcomesString + " NA "
            
        outcomesString = outcomesString + "</outcomes>\n"
        
        self.xmlString = self.xmlString + outcomesString
        
        
        # TBA is appended if chief examiner list is null
        
        chiefExaminersString = "<chief_examiners>"
        
        if len(self.examiner) > 0:
            chiefExaminersString += "\n"
            for each in self.examiner:
                chiefExaminersString = chiefExaminersString +"<chief_examiner>" +each+"</chief_examiner>"
        else:
            chiefExaminersString = chiefExaminersString + " TBA "
            
        chiefExaminersString = chiefExaminersString + "</chief_examiners>\n"
        
        self.xmlString = self.xmlString + chiefExaminersString
        
        self.xmlString = self.xmlString + "</unit>\n"
        
        #self.writeXML()
        
    
    def writeXML(self):
        # The class variable is written to a file
        f = open("29567467.xml", "w")
        f.write(self.xmlString + "</units>")
        f.close()
        
    def createJson(self):
        
        # For synopsis, outcome and requirement, we have seen that "&#38;" is replaced by "&" and "&gt;" is 
        # replaced by ">" in the final json
        if len(self.synopsis) != 0:
            self.synopsis[0] = re.sub(r"&#38;", "&", self.synopsis[0])
            self.synopsis[0] = re.sub(r"&gt;", ">", self.synopsis[0])
                      
        prequisiteDict = {}
        prequisiteDict['pre_requistic'] = self.prequisites
        if(len(self.prequisites) == 0):
            prequisiteDict = "NA"
        if(len(self.prequisites) == 1):
            prequisiteDict['pre_requistic'] = self.prequisites[0]
            
        prohibitorsDict = {}
        prohibitorsDict['prohibision'] = self.prohibitors
        if(len(self.prohibitors) == 0):
            prohibitorsDict = "NA"
        if(len(self.prohibitors) == 1):
            prohibitorsDict['prohibision'] = self.prohibitors[0]
            
        # For synopsis, outcome and requirement, we have seen that "&#38;" is replaced by "&" and "&gt;" is 
        # replaced by ">" in the final json
        corrected_req = []
        for each in self.requirements:
            each = re.sub(r"&#38;", "&", each)
            each = re.sub(r"&gt;", ">", each)
            corrected_req.append(each)
            
        requirementsDict = {}
        requirementsDict['requirement'] = corrected_req
        if(len(self.requirements) == 0):
            requirementsDict = "NA"   
        if(len(self.requirements) == 1):
            requirementsDict['requirement'] = self.requirements[0]
            
        examinerDict = {}
        examinerDict['chief_examiner'] = self.examiner
        if(len(self.examiner) == 0):
            examinerDict = "TBA" 
        if(len(self.examiner) == 1):
            examinerDict['chief_examiner'] = self.examiner[0]
        
        # For synopsis, outcome and requirement, we have seen that "&#38;" is replaced by "&" and "&gt;" is 
        # replaced by ">" in the final json
        corrected_outcome = []
        for each in self.outcomes:
            each = re.sub(r"&#38;", "&", each)
            each = re.sub(r"&gt;", ">", each)
            corrected_outcome.append(each)
            
        outcomeDict = {}
        outcomeDict['outcome'] = corrected_outcome
        if(len(self.outcomes) == 0):
            outcomeDict = "NA" 
        if(len(self.outcomes) == 1):
            outcomeDict['outcome'] = self.outcomes[0]            
        if(len(self.synopsis) == 0):
            self.synopsis = "NA"  

        self.unit_list.append({  
            '@id': self.unit_code,
            'title': self.title[1:],
            'synopsis': self.synopsis[0],
            'pre_requistics': prequisiteDict,
            'prohibisions': prohibitorsDict,
            'requirements' : requirementsDict,
            'outcomes' : outcomeDict,
            'chief_examiners' : examinerDict
        })
        unit ={}
        unit['unit'] = self.unit_list
        self.unit['units'] = unit
    
    # Function to write the dictionary to a json file using Json package
    def writeJson(self):
        
        with open('29567467.json', 'w') as outfile:  
            json.dump(self.unit, outfile, indent=5)
        

In [6]:
u = UnitDetailsExtraction("29567467.txt")

## Summary
* Used regex to extract data from a semi structured file format
* Used json package to write data into required format
* Understood about the json and xml file formats